In [ ]:
%pip install nltk
%pip install openai
%pip install pdfminer.six
%pip install text_chunker
%pip install azure-search-documents

In [1]:
import os
import time
import uuid
from pdfminer.high_level import extract_text
from openai import AzureOpenAI
from text_chunker import TextChunker
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

In [ ]:
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
key = os.getenv("AZURE_SEARCH_API_KEY")

TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"
chunck_size = 8191
index_name = "faq-bc-arquivo"

chukner = TextChunker(maxlen=chunck_size)
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

# inicialização do cliente Azure OpenAI
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview"
)


def read_file(file_path):
    if file_path.endswith('.pdf'):
        return extract_text(file_path)
    else:
        with open(file_path, 'r') as file:
            text = file.read()
        return text

def generate_embeddings(text):
    start_time = time.perf_counter()
    embeddings = client.embeddings.create(input = [text], model=TEXT_EMBEDDING_ADA_002).data[0].embedding
    elapsed_time = time.perf_counter() - start_time
    return embeddings, elapsed_time

def upload_document(document):
    result = search_client.upload_documents(documents=[document])
    print("Upload of new document succeeded: {}".format(result[0].succeeded))

def merge_document():
    result = search_client.merge_documents(documents=[{"hotelId": "1000", "rating": 4.5}])
    print("Merge into new document succeeded: {}".format(result[0].succeeded))


def delete_document():
    result = search_client.delete_documents(documents=[{"hotelId": "1000"}])
    print("Delete new document succeeded: {}".format(result[0].succeeded))

def get_document(id):
    result = search_client.get_document(id)

    print("Details:")
    print("Filepath: {}".format(result["filepath"]))
    print("Title: {}".format(result["title"]))
    print("URL: {}".format(result["url"]))
    print("Content: {}".format(result["content"]))
    print("ContentVector: {}".format(result["contentVector"]))

def query(query_string):
    results = search_client.search(search_text=query_string, top=5)
    print("Search results:")
    for result in results:
        print("Filepath: {}".format(result["filepath"]))
        print("Title: {}".format(result["title"]))
        print("URL: {}".format(result["url"]))
        print("Content: {}".format(result["content"]))
        print("ContentVector: {}".format(result["contentVector"]))
        print("====================================")

# funcao para dividir o texto em chunks
def chunk_text(text):
    chunks = chukner.chunk(text)
    return chunks

# funcao para ler um arquivo e dividir em chunks
def chunk_file(file_path):
    text = read_file(file_path)
    chunks = chunk_text(text)
    return chunks

# funcao para gerar embeddings de um arquivo
def generate_embeddings_file(file_path, model):
    chunks = chunk_file(file_path)
    embeddings = []
    elapsed_time = 0
    for chunk in chunks:
        chunk_embeddings, chunk_elapsed_time = generate_embeddings(chunk, model)
        embeddings.append(chunk_embeddings)
        elapsed_time += chunk_elapsed_time
    return embeddings, elapsed_time

# funcao para gerar um uuid para o arquivo
def generate_uuid():
    return str(uuid.uuid4())


def index_file(file_path):
    title = os.path.basename(file_path)
    chunks = chunk_file(file_path)
    last_updated = time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime(os.path.getmtime(file_path)))
    chunk_id = 0
    for chunk in chunks:
        chunk_id += 1
        embeddings, elapsed_time = generate_embeddings(chunk)
        document = {
            "content": chunk,
            "filepath": file_path,
            "title": title,
            "url": "file://" + file_path,
            "id": generate_uuid(),
            "chunk_id": f"{chunk_id}",
            "last_updated": last_updated,
            "contentVector": embeddings
        }
        upload_document(document)
        print(document)
    print(f"File indexed successfully! Total chunks: {chunk_id} - Elapsed time: {elapsed_time} seconds")
    return True


def index_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                file_path = os.path.join(root, file)
                index_file(file_path)
    return True

#arquivo = "/Users/rodam/Downloads/Manual beneficios.pdf"
#index_file(arquivo)
query_string = "manual de beneficios"
query(query_string)